## Ques2: Based on PPMI matrix

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("./IMDB Dataset.csv")
print(df)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


In [3]:
df_new = df
df_new = df_new.drop(['sentiment'], axis=1) # sentiment dropped
df_new = df_new[:1000] # first 1000 reviews
print(df_new)

                                                review
0    One of the other reviewers has mentioned that ...
1    A wonderful little production. <br /><br />The...
2    I thought this was a wonderful way to spend ti...
3    Basically there's a family where a little boy ...
4    Petter Mattei's "Love in the Time of Money" is...
..                                                 ...
995  Nothing is sacred. Just ask Ernie Fosselius. T...
996  I hated it. I hate self-aware pretentious inan...
997  I usually try to be professional and construct...
998  If you like me is going to see this in a film ...
999  This is like a zoology textbook, given that it...

[1000 rows x 1 columns]


In [4]:
#Create a list of list named Corpus whose each list is a review (document) from IMDB dataset
corpus=[]
for i in range(len(df_new)):
    corpus.append(df_new['review'][i])

In [5]:
corpus[0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [6]:
# remove non-aplha characters and converting words to lower case in each document
corpus_final=[]
for i in range(len(corpus)):
    corpus_final.append(" ".join([word.lower() for word in corpus[i].split() if word.isalpha()]))

In [7]:
# Creating binary bow vector as dictionary of dictionary
# Each external dictionary will have key as document number and internal dictionaries have the
# keys as the word present in the document
document_frequency={}
for i in range(len(corpus_final)):
    word_frequency={}
    for word in corpus_final[i].split():
        word_frequency[word]=1
    document_frequency[i]=word_frequency

In [8]:
document_frequency[0]

{'one': 1,
 'of': 1,
 'the': 1,
 'other': 1,
 'reviewers': 1,
 'has': 1,
 'mentioned': 1,
 'that': 1,
 'after': 1,
 'watching': 1,
 'just': 1,
 'oz': 1,
 'episode': 1,
 'be': 1,
 'they': 1,
 'are': 1,
 'as': 1,
 'this': 1,
 'is': 1,
 'exactly': 1,
 'what': 1,
 'happened': 1,
 'with': 1,
 'first': 1,
 'thing': 1,
 'struck': 1,
 'me': 1,
 'about': 1,
 'was': 1,
 'its': 1,
 'brutality': 1,
 'and': 1,
 'unflinching': 1,
 'scenes': 1,
 'which': 1,
 'set': 1,
 'in': 1,
 'right': 1,
 'from': 1,
 'word': 1,
 'trust': 1,
 'not': 1,
 'a': 1,
 'show': 1,
 'for': 1,
 'faint': 1,
 'hearted': 1,
 'or': 1,
 'pulls': 1,
 'no': 1,
 'punches': 1,
 'regards': 1,
 'to': 1,
 'sex': 1,
 'classic': 1,
 'use': 1,
 'called': 1,
 'nickname': 1,
 'given': 1,
 'oswald': 1,
 'maximum': 1,
 'security': 1,
 'state': 1,
 'it': 1,
 'focuses': 1,
 'mainly': 1,
 'on': 1,
 'emerald': 1,
 'an': 1,
 'experimental': 1,
 'section': 1,
 'prison': 1,
 'where': 1,
 'all': 1,
 'cells': 1,
 'have': 1,
 'glass': 1,
 'fronts': 1,
 

In [9]:
#Converting dicitonaries of dicitonaries into DataFrame; 
# setting NaN as 0 (for terms which are not present in particular document)
bow = pd.DataFrame(document_frequency)
bow.fillna(0,inplace=True)
bow.shape

(14719, 1000)

In [10]:
cooccurrence_matrix = bow.values.dot(bow.values.T)

In [11]:
np.fill_diagonal(cooccurrence_matrix,0)

In [12]:
cooccurrence_matrix.shape

(14719, 14719)

In [13]:
cooccurrence_matrix

array([[  0., 496., 500., ...,   1.,   1.,   1.],
       [496.,   0., 950., ...,   1.,   1.,   1.],
       [500., 950.,   0., ...,   1.,   1.,   1.],
       ...,
       [  1.,   1.,   1., ...,   0.,   1.,   1.],
       [  1.,   1.,   1., ...,   1.,   0.,   1.],
       [  1.,   1.,   1., ...,   1.,   1.,   0.]])

In [16]:
import scipy.sparse

In [17]:
def PPMI(C):
    # Total count.
    Z = float(C.sum())
    # Sum each row (along columns).
    Zr = np.array(bow.sum(axis=1), dtype=np.float64).flatten()
    
    # Get indices of relevant elements.
    ii, jj = C.nonzero()  # row, column indices
    Cij = np.array(C[ii,jj], dtype=np.float64).flatten()

    # PMI equation.
    pmi = np.log(Cij * 1000 / (Zr[ii] * Zr[jj]))

    # Truncate to positive only.
    ppmi = np.maximum(0, pmi)  # take positive only
    # Re-format as sparse matrix.
    ret = scipy.sparse.csc_matrix((ppmi, (ii,jj)), shape=C.shape, dtype=np.float64)
    return ret

In [18]:
ppmi_matrix = PPMI(cooccurrence_matrix)

In [19]:
print(ppmi_matrix)

  (1, 0)	0.037061433173913166
  (2, 0)	0.00402006966289006
  (3, 0)	0.21284674583684154
  (4, 0)	0.0
  (5, 0)	0.13921588613386576
  (6, 0)	0.0780447050108633
  (7, 0)	0.06091582274789613
  (8, 0)	0.18644914755796427
  (9, 0)	0.0
  (10, 0)	0.04594145962644595
  (11, 0)	0.6911491778972721
  (12, 0)	0.20931109100453393
  (13, 0)	0.10002361172963863
  (14, 0)	0.07210996949104888
  (15, 0)	0.07876335398178533
  (16, 0)	0.10466671924457756
  (17, 0)	0.02549135883512455
  (18, 0)	0.028773656004080547
  (19, 0)	0.38366447814931165
  (20, 0)	0.09899330888448937
  (21, 0)	0.0
  (22, 0)	0.08189997221917465
  (23, 0)	0.048432850964218815
  (24, 0)	0.16308174769677558
  (25, 0)	0.28568406978910793
  :	:
  (12616, 14718)	6.214608098422191
  (12851, 14718)	6.214608098422191
  (13018, 14718)	6.214608098422191
  (13292, 14718)	6.214608098422191
  (13909, 14718)	5.809142990314028
  (13959, 14718)	5.809142990314028
  (14132, 14718)	6.214608098422191
  (14700, 14718)	6.907755278982137
  (14701, 14718)	6.9

In [20]:
ppmi = ppmi_matrix.toarray()

In [21]:
ppmi_df = pd.DataFrame(ppmi, index=bow.index, columns=bow.index, dtype='float')
ppmi_df

,one,of,the,other,reviewers,has,mentioned,that,after,watching,...,cameraman,microphones,faintest,handguns,hesitation,offices,intuition,stubborn,masking,planes
one,0.000000,0.037061,0.004020,0.212847,0.000000,0.139216,0.078045,0.060916,0.186449,0.000000,...,0.691149,0.691149,0.691149,0.691149,0.691149,0.691149,0.691149,0.691149,0.691149,0.691149
of,0.037061,0.000000,0.001816,0.030425,0.047092,0.026312,0.047092,0.012742,0.023781,0.000000,...,0.047092,0.047092,0.047092,0.047092,0.047092,0.047092,0.047092,0.047092,0.047092,0.047092
the,0.004020,0.001816,0.000000,0.006018,0.006018,0.003444,0.006018,0.003417,0.001399,0.006018,...,0.006018,0.006018,0.006018,0.006018,0.006018,0.006018,0.006018,0.006018,0.006018,0.006018
other,0.212847,0.030425,0.006018,0.000000,0.166055,0.267423,0.437988,0.100303,0.329380,0.078433,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
reviewers,0.000000,0.047092,0.006018,0.166055,0.000000,0.384560,1.783791,0.261365,0.275095,1.056511,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
offices,0.691149,0.047092,0.006018,0.000000,0.000000,0.944176,0.000000,0.261365,0.000000,0.000000,...,6.907755,6.907755,6.907755,6.907755,6.907755,0.000000,6.907755,6.907755,6.907755,6.907755
intuition,0.691149,0.047092,0.006018,0.000000,0.000000,0.944176,0.000000,0.261365,0.000000,0.000000,...,6.907755,6.907755,6.907755,6.907755,6.907755,6.907755,0.000000,6.907755,6.907755,6.907755
stubborn,0.691149,0.047092,0.006018,0.000000,0.000000,0.944176,0.000000,0.261365,0.000000,0.000000,...,6.907755,6.907755,6.907755,6.907755,6.907755,6.907755,6.907755,0.000000,6.907755,6.907755
masking,0.691149,0.047092,0.006018,0.000000,0.000000,0.944176,0.000000,0.261365,0.000000,0.000000,...,6.907755,6.907755,6.907755,6.907755,6.907755,6.907755,6.907755,6.907755,0.000000,6.907755


### Find word embeddings of the PPMI matrix of dimensionality 500 using SVD decomposition

In [22]:
import cupy as cp

In [23]:
matrix = cp.array(ppmi)

### Find eigen values, eigen vectors of ppmi matrix

In [25]:
eig_val, eig_vec = cp.linalg.eigh(matrix)

In [26]:
eig_val

array([-706.23774456, -148.93911982, -139.42984955, ...,  901.35557275,
        944.60902141, 1895.01881295])

In [27]:
idx = eig_val.argsort()[::-1]
eig_val = eig_val[idx]
eig_vec = eig_vec[:,idx]

In [28]:
eig_val500 = eig_val[:500]

In [29]:
eig_vec500 = eig_vec[:500,:]

### S,U,V matrices of SVD
### A = U*S*V 

In [32]:
S = cp.zeros([500,500])
cp.fill_diagonal(S,cp.sqrt(eig_val500))

In [33]:
V = eig_vec500.T

In [34]:
print(eig_val500.shape, S.shape, V.shape)

(500,) (500, 500) (14719, 500)


In [35]:
sigma = eig_val500
sigma.shape

(500,)

In [36]:
print(V.shape, matrix.shape)

(14719, 500) (14719, 14719)


In [37]:
U = (1/sigma)*(cp.dot(matrix,V))

In [38]:
U.shape

(14719, 500)

In [39]:
import cudf as cd

In [40]:
word_embedd = cd.DataFrame(U, index=bow.index, dtype='float')
word_embedd

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
one,0.000192,0.000053,-0.000152,-0.000185,5.927282e-04,0.000113,0.000083,0.000245,-0.000122,0.000416,...,-0.002563,0.003921,0.000749,0.001145,0.005507,0.005597,-0.000986,0.008349,-0.002721,-0.001532
of,-0.000001,0.000026,0.000027,-0.000034,-2.011412e-06,0.000034,-0.000052,0.000001,-0.000041,-0.000007,...,-0.000603,0.000127,-0.000280,0.000287,0.000415,0.000370,0.000008,0.000386,-0.000408,0.000036
the,0.000001,0.000003,0.000002,-0.000006,-2.537063e-07,0.000002,-0.000007,0.000004,-0.000003,-0.000003,...,-0.000090,0.000007,-0.000056,0.000038,0.000055,0.000057,0.000004,0.000033,-0.000038,-0.000005
other,-0.000175,0.000621,0.000589,0.001025,3.522024e-04,-0.000675,0.000103,-0.000316,-0.000466,-0.000106,...,-0.008236,0.006073,-0.002180,0.001739,0.005572,0.007736,0.002005,-0.007892,0.001705,-0.001042
reviewers,-0.000116,-0.000012,-0.000017,-0.000160,-2.297652e-05,0.000735,0.000242,0.000455,0.000247,-0.001016,...,0.002587,-0.000999,0.002871,0.012662,0.002673,0.001782,0.005388,0.001250,-0.005056,0.002438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
offices,-0.000342,-0.000359,-0.000071,-0.000376,-3.320914e-04,0.000465,-0.000725,-0.000151,-0.000989,-0.000343,...,-0.009122,-0.010097,0.004061,0.009686,0.003570,-0.009534,-0.000859,-0.008262,0.005201,0.011671
intuition,-0.000297,-0.000367,-0.000072,-0.000016,-3.107730e-04,0.000455,-0.000428,-0.000171,-0.000858,-0.000659,...,-0.007872,-0.008789,0.004097,0.009418,0.002902,-0.013868,-0.000644,-0.008939,0.005385,0.008518
stubborn,-0.000488,-0.000379,-0.000074,-0.000562,-3.696075e-04,-0.000110,-0.000104,-0.000320,-0.000984,-0.000232,...,-0.008005,-0.004739,0.003783,0.009490,0.003888,-0.014196,0.000612,-0.007868,0.006026,0.012593
masking,-0.000208,-0.000349,-0.000070,0.000021,-2.819753e-04,0.000571,-0.000428,-0.000031,-0.000572,-0.000318,...,-0.005542,-0.008691,0.004334,0.009835,0.003445,-0.014220,-0.001426,-0.008710,0.005656,0.011551
